In [1]:
# SIMPLE GENRE-BASED MOVIE RECOMMENDATION SYSTEM

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 1. LOAD MOVIES DATASET
movies = pd.read_csv("movies.csv")        # columns: movieId, title, genres
links  = pd.read_csv("links.csv")         # columns: movieId, imdbId, tmdbId

# Clean genres
movies["genres"] = movies["genres"].fillna("").str.replace("|", " ")

# Merge IMDb/TMDB links
movies = movies.merge(links, on="movieId", how="left")

# Create IMDb URL column
movies["imdb_url"] = movies["imdbId"].apply(
    lambda x: f"https://www.imdb.com/title/tt{int(x):07d}/" if pd.notnull(x) else ""
)

# 2. TF-IDF VECTORIZATION ON GENRES
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(movies["genres"])

# Compute similarity
genre_similarity = cosine_similarity(tfidf_matrix)


# 3. GENRE-BASED MOVIE RECOMMENDATION FUNCTION
def recommend_by_genre(selected_genre, top_n=10):

    selected_genre = selected_genre.lower().strip()

    # Check if genre exists anywhere
    genre_matches = movies[movies["genres"].str.lower().str.contains(selected_genre)]

    if genre_matches.empty:
        return f" Genre '{selected_genre}' not found."

    # Take the first matched movie to compute similarity
    idx = genre_matches.index[0]

    # Sort similarity scores
    scores = list(enumerate(genre_similarity[idx]))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)

    # Top recommended movies
    top_indices = [i[0] for i in scores[1:top_n+1]]
    recommended = movies.iloc[top_indices][["title", "genres", "imdb_url"]]

    return recommended


# 4. RUN SYSTEM
if __name__ == "__main__":
    print("\n Genre-Based Movie Recommendation System Ready!")
    genre = input("Enter Genre (example: Action, Comedy, Horror): ")

    print("\n Recommended Movies:\n")
    result = recommend_by_genre(genre)

    if isinstance(result, str):
        print(result)
    else:
        for i, row in result.iterrows():
            print(f"{row['title']}  |  {row['genres']}  |  {row['imdb_url']}")



 Genre-Based Movie Recommendation System Ready!

 Recommended Movies:

Tales from the Crypt Presents: Bordello of Blood (1996)  |  Comedy Horror  |  https://www.imdb.com/title/tt0117826/
Gremlins (1984)  |  Comedy Horror  |  https://www.imdb.com/title/tt0087363/
Gremlins 2: The New Batch (1990)  |  Comedy Horror  |  https://www.imdb.com/title/tt0099700/
Attack of the Killer Tomatoes! (1978)  |  Comedy Horror  |  https://www.imdb.com/title/tt0080391/
Pet Sematary II (1992)  |  Comedy Horror  |  https://www.imdb.com/title/tt0105128/
Idle Hands (1999)  |  Comedy Horror  |  https://www.imdb.com/title/tt0138510/
Arachnophobia (1990)  |  Comedy Horror  |  https://www.imdb.com/title/tt0099052/
Little Shop of Horrors, The (1960)  |  Comedy Horror  |  https://www.imdb.com/title/tt0054033/
Bride of Re-Animator (1990)  |  Comedy Horror  |  https://www.imdb.com/title/tt0099180/
Class of Nuke 'Em High (1986)  |  Comedy Horror  |  https://www.imdb.com/title/tt0090849/
